# Export VietNet Food Data to Text Format

Xuất toàn bộ dữ liệu VietNet Food thành định dạng văn bản:

```
<number_of_tab> (<level>) {<lemma1>, <lemma2>, ...} [<synset_id>]: <definition>
```


In [1]:
# Import và xử lí dữ liệu
import sys
import os

# Add components to path
sys.path.append(os.path.join(os.getcwd(), 'components'))

from components.utils_wn import *
from components.class_Node import *
from components.class_NodeFamily import *
import wn

# Initialize lexicon
lexicon = wn.Wordnet('vietnet-food:1.0')

# Get all root synsets (synsets with no hypernyms)
all_synsets = list(lexicon.synsets())
root_synsets = [syn for syn in all_synsets if not syn.hypernyms()]

print(f"Tổng số synsets: {len(all_synsets)}")
print(f"Số lượng root synsets: {len(root_synsets)}")

# Build tree with hyponym relationship and maximum depth
families = NodeFamily(root_synsets, 'hyponym', max_recursive=100)

# Function to export nodes to text with proper formatting
def export_to_text(nodes, output_file='vietnet_food_export.txt'):
    """
    Export node tree to text file with format:
    <tabs> (<level>) {lemmas} [synset_id]: definition
    """
    lines = []
    
    def traverse_node(node, depth=0):
        # Calculate level from root (node._level is absolute level from root)
        level = node._level - 1  # Convert to 0-based level
        
        # Create tabs for indentation
        tabs = '\t' * level
        
        # Get lemmas
        lemmas = node._lemmas
        
        # Get synset ID
        synset_id = node._synset.id
        
        # Get definition
        definition = node._definition
        
        # Format: <tabs> (<level>) {lemmas} [synset_id]: definition
        line = f"{tabs}({level}) {{{lemmas}}} [{synset_id}]: {definition}"
        lines.append(line)
        
        # Recursively traverse children
        if node.children:
            for child in node.children:
                traverse_node(child, depth + 1)
    
    # Traverse all root nodes
    for node in nodes:
        traverse_node(node)
    
    # Write to file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(lines))
    
    print(f"\n✅ Đã xuất {len(lines)} dòng dữ liệu vào file: {output_file}")
    print(f"\n📄 Preview (20 dòng đầu):")
    print("=" * 80)
    for i, line in enumerate(lines[:20]):
        print(line)
    print("=" * 80)
    
    return lines

# Export data
exported_lines = export_to_text(families.nodes, 'vietnet_food_export.txt')

print(f"\n📊 Thống kê:")
print(f"   - Tổng số dòng đã xuất: {len(exported_lines)}")
print(f"   - File được lưu tại: vietnet_food_export.txt")
print(f"\n💡 Bạn có thể mở file này bằng Word hoặc Google Docs để xem toàn bộ dữ liệu.")


Tổng số synsets: 590
Số lượng root synsets: 2

✅ Đã xuất 590 dòng dữ liệu vào file: vietnet_food_export.txt

📄 Preview (20 dòng đầu):
(0) {thức ăn, chất} [vietnet-food-00045020-n]: thứ có chứa chất dinh dưỡng và có thể tiêu hóa được, dùng để nuôi sống con người và động vật, nói chung
	(1) {thực phẩm} [vietnet-food-00045060-n]: các thứ dùng để chế biến thành món ăn, như thịt, cá, trứng, v.v. [nói khái quát]; phân biệt với lương thực
		(2) {nông sản, nông phẩm} [vietnet-food-00034540-n]: sản phẩm nông nghiệp, như gạo, thịt, trứng, rau, hoa quả, v.v. [nói khái quát]
			(3) {súc sản} [vietnet-food-00040488-n]: sản phẩm thịt gia súc
				(4) {cầy, giò} [vietnet-food-00006363-n]: chó, về mặt để ăn thịt
				(4) {nạc} [vietnet-food-00030509-n]: thịt không có mỡ
				(4) {thăn} [vietnet-food-00042705-n]: phần thịt toàn nạc được lọc ra ở phần lưng của súc vật khi mổ thịt
				(4) {nọng} [vietnet-food-00034330-n]: khoanh thịt cắt ra ở cổ gia súc giết thịt [thường là trâu, bò, lợn]
				(4) {ba chỉ, b

Skipping vietnet-food:1.0 (VietNet Vietnamese Food Lexicon (All)); already addedd (thủ công).xml

